# *Install and Import Packages*

In [1]:
pip install pulp

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from numpy.random import default_rng
import time
import heapq
from pulp import GLPK
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# *change date to number*

In [3]:
# It is necessary to change the date column to number for calculation
def TimetoNum(x):
    """
    it Convert the time column to number for easier analysis.
    
    Argument:
    x -- Name of the date coulum in the data which needs to change to number.
        
    Returns:
    y -- A list of number
    """
    # Change it from format '%m/%d/%Y %H:%M' to format '%Y-%m-%d %H:%M' then convert it to number
    utc_time = datetime.strptime(x, '%m/%d/%Y %H:%M')
    x = datetime.strftime(utc_time, '%Y-%m-%d %H:%M')
    y = datetime.fromisoformat(x).timestamp()
    return y

# *Fitness Function*

In [4]:
def Fitness(dataset, individual):
        """
        It calculates the fitness of each individual.
        
        Arguments:
        dataset -- Clean dataset of vessels which are going to schedule
        individual -- One individual of the population
        
        Returns:
        fitness -- The fitness value for the input individual
        scheme -- The optimum scheduling scheme for the individual according to the input individual
        """
        # Define the optimization model
        model = LpProblem(name="small-problem", sense=LpMinimize)  

        # Define the scheduled scheme variable.
        Scheduled = {i: LpVariable(name=('Scheduled %i' % i), 
                                lowBound=dataset["PilotOrderTime"][i]) for i in range(len(dataset))}

        # Define the binary variables for activating one restiction between two contradict restrictions at the same time
        x = {j: LpVariable(name=f"x{j}", lowBound=0, upBound = 1, cat="Integer") for j in range(100)}
        y = {o: LpVariable(name=f"y{o}", lowBound=0, upBound = 1, cat="Integer") for o in range(100)}

        # Axuiliary variables and parameters
        l=0            
        k=0
        kk=0
        m=10000000

        # Apply and check all the restrictions in the model.
        for i in individual:
            model += Scheduled[i]>= dataset["PilotOrderTime"][i]
            model += Scheduled[i]<= 1533083200+m*y[kk]
            model += Scheduled[i]>= 1533084400+m*(y[kk]-1)
            kk=kk+1
            l=l+1
            if (dataset["d4"][i]>0):
                if ((dataset["Type"][i]== 2 and dataset["Beam"][i]>=120)or (dataset["Beam"][i]>=150)):
                    model += Scheduled[i]>= 1533104500
                    model += Scheduled[i]<= 1533146400
        
            for j in individual[l:]:
                if (dataset["Direction"][i]==1 and (dataset["Direction"][j]!= dataset["Direction"][i])):
                    if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]> dataset["d2"][i])):
                        LB1= Scheduled[i]+ (dataset["d1"][i]/dataset["Speed"][i])*60*60
                        LB2= (dataset["d2"][j]/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d2"][i]/dataset["Speed"][i])*60*60
                        UB2= ((dataset["d2"][j]-dataset["d2"][i])/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
                                                
                        
                    if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]<= dataset["d2"][i])):
                        LB1= Scheduled[i]+(dataset["d1"][i]/dataset["Speed"][i])*60*60
                        LB2= (dataset["d2"][j]/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d2"][j]/dataset["Speed"][i])*60*60
                        UB2= 0
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
                        
                    if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]> dataset["d4"][i])):
                        LB1= Scheduled[i]+ (dataset["d3"][i]/dataset["Speed"][i])*60*60
                        LB2= (dataset["d4"][j]/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d4"][i]/dataset["Speed"][i])*60*60
                        UB2= ((dataset["d4"][j]-dataset["d4"][i])/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
                                                
                        
                    if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]<= dataset["d4"][i])):
                        LB1= Scheduled[i]+(dataset["d3"][i]/dataset["Speed"][i])*60*60
                        LB2= (dataset["d4"][j]/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d4"][j]/dataset["Speed"][i])*60*60
                        UB2= 0
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1     
                                
                if (dataset["Direction"][i]==0 and (dataset["Direction"][j]!= dataset["Direction"][i])):
                    if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]> dataset["d2"][i])):
                        LB1= Scheduled[i]
                        LB2= ((dataset["d1"][j]+dataset["d2"][i])/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d2"][i]/dataset["Speed"][i])*60*60
                        UB2= (dataset["d1"][j]/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
                                
                        
                    if ((dataset["Beam"][i]+dataset["Beam"][j]>= 272) and (dataset["d2"][i]>0 and dataset["d2"][j]<= dataset["d2"][i])):
                        LB1= Scheduled[i]+((dataset["d2"][i]-dataset["d2"][j])/dataset["Speed"][i])*60*60
                        LB2= ((dataset["d1"][j]+dataset["d2"][j])/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d2"][j]/dataset["Speed"][i])*60*60
                        UB2= (dataset["d1"][j]/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1

                    if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]> dataset["d4"][i])):
                        LB1= Scheduled[i]
                        LB2= ((dataset["d3"][j]+dataset["d4"][i])/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d4"][i]/dataset["Speed"][i])*60*60
                        UB2= (dataset["d3"][j]/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
                                
                        
                    if ((dataset["Beam"][i]>=106 or dataset["Beam"][j]>= 106) and (dataset["d4"][i]>0 and dataset["d4"][j]<= dataset["d4"][i])):
                        LB1= Scheduled[i]+((dataset["d4"][i]-dataset["d4"][j])/dataset["Speed"][i])*60*60
                        LB2= ((dataset["d3"][j]+dataset["d4"][j])/dataset["Speed"][j])*60*60
                        UB1= LB1+ (dataset["d4"][j]/dataset["Speed"][i])*60*60
                        UB2= (dataset["d3"][j]/dataset["Speed"][j])*60*60
                        model+= Scheduled[j]<=(LB1-LB2)+m*x[k] 
                        model+= Scheduled[j]>=(UB1-UB2)+m*(x[k]-1)
                        k=k+1
        
        xy= lpSum(Scheduled)
        yx= lpSum(dataset["PilotOrderTime"])

        # Define the Objective Function.
        model += xy-yx

        # Solve the optimization model.
        status = model.solve()
        fitness= model.objective.value()
        scheme= model.variables()

        return (fitness, scheme)


# *Mutation*

In [5]:
def mutation(dataset, individual):
        """
        It performs the mutation technique on individuals
        
        Argument:
        dataset -- Clean dataset of vessels which are going to schedule
        individual -- One individual of the population
        
        Returns:
        y -- Individual after mutation
        """
        rand1= np.random.randint(0, len(dataset))
        rand2= np.random.randint(rand1, len(dataset))
        y= individual.copy()
        y[rand1]= individual[rand2].copy()
        y[rand2]= individual[rand1].copy()
        return y

# *Crossover*

In [6]:
def crossover(dataset, selection):
        """
        It performs two-point crossover technique.
        
        Argument:
        dataset -- Clean dataset of vessels which are going to schedule.
        selection -- Two individuals for crossover techniques
        
        Returns:
        offspring -- New offsprings
        """
        # First Offspring
        offspring=[]

        rand1= np.random.randint(1, len(dataset)-1)
        rand2= np.random.randint(rand1+1, len(dataset))

        y= selection[0].copy()
        y[0:(len(dataset)-rand2)]= selection[1][rand2:]
        y[len(dataset)-rand1:]= selection[1][:rand1]

        ind = 0
        for i in range (20):
            if ((i not in selection[1][rand2:]) and (i not in selection[1][:rand1])):
                y[(len(dataset)-rand2)+ind] = i
                ind += 1

        offspring.append(y)

        # Second Offspring

        y= selection[1].copy()
        y[0:(len(dataset)-rand2)]= selection[0][rand2:]
        y[len(dataset)-rand1:]= selection[0][:rand1]

        ind = 0      
        for i in range (20):
            if ((i not in selection[0][rand2:]) and (i not in selection[0][:rand1])):
                y[(len(dataset)-rand2)+ind] = i
                ind += 1
                        
        offspring.append(y)

        return offspring


# *Import Data*

In [7]:
dataset = pd.read_csv('Real_20.csv')

# *Change date to number*

In [8]:
dataset["PilotOrderTime"]=dataset["PilotOrderTime"].apply(TimetoNum)

# *Genetic Algorithm*

In [ ]:
start_time= time.clock()
sol_glo= 10000000000000
""" Genetic algorithm is performed in this section and the best found solution is presented. """

# Population Generation
for r in range(5):
    rng = default_rng()
    population=[]
    for i in range(10):
        numbers = np.array(rng.choice(len(dataset), size=len(dataset), replace=False))
        population.append(numbers)

    
    for j in range(5):
        # Cost of individuals are calculated.
        cost= []
        for i in range(len(population)):
            Scheduled= dataset["PilotOrderTime"].copy()
            cost.append(Fitness(dataset,population[i])[0])

        # The minimum cost is found.
        sol_loc= min(cost)
        if sol_loc<= sol_glo:
            sol_glo= sol_loc
            sol_index= cost.index(min(cost))
            sol= population[sol_index]
            final_scheme= Fitness(dataset,population[sol_index])[1]
        
        # Roulette Wheel
        reciprocal = []
        total = 0
        for i in range(len(population)):
            total += Fitness(dataset, population[i])[0]

        for pop in range(len(population)):
            reciprocal.append((Fitness(dataset,population[pop])[0])/total)  
        gg= np.cumsum(reciprocal)
        
        # Selection and crossover
        Q= []
        for zz in range(4):
            selection = []
            cross = []
            for j in range(2):
                a = np.random.rand()
                for i in range(10):
                    if a <= gg[i]:
                        selection.append(list(population[i]))
                        break
            cross = crossover(dataset, selection)
            Q.append(cross[0])
            Q.append(cross[1])

        # The best two individual are kept.
        a= heapq.nsmallest(2, ((k, yy) for yy, k in enumerate(cost)))
        Z=[]
        for i in range(len(a)):
            Z.append(list(population[a[i][1]]))

        for i in range(2):
            Q.append(Z[i])

        population = Q

        

duration= time.clock()- start_time

# *Final Solution*

In [ ]:
""" The final Solution """

print(sol_glo)
print(sol)
print(dataset["PilotOrderTime"][7])
for var in final_scheme:
    print(f"{var.name}: {var.value()}")

In [11]:
print(duration)

23.485682999999998
